In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from sklearn.model_selection import train_test_split
from shutil import copyfile

mainDataDir='/kaggle/input/multi-cancer/Multi Cancer/Lung and Colon Cancer'
colonAcaDir=os.path.join(mainDataDir,'colon_aca')
colonBntDir=os.path.join(mainDataDir,'colon_bnt')
lungAcaDir=os.path.join(mainDataDir,'lung_aca')
lungBntDir=os.path.join(mainDataDir,'lung_bnt')
lungSccDir=os.path.join(mainDataDir,'lung_scc')

In [3]:
trainDir = '/kaggle/working/training_data'
valDir = '/kaggle/working/validation_data'

In [4]:
os.makedirs(trainDir,exist_ok=True)
os.makedirs(valDir,exist_ok=True)

In [5]:
def split_and_copy(class_dir, train_output_dir, val_output_dir):
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith(('jpg', 'jpeg', 'png'))]

    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(train_output_dir, exist_ok=True)
    os.makedirs(val_output_dir, exist_ok=True)

    for img in train_images:
        copyfile(img, os.path.join(train_output_dir, os.path.basename(img)))

    for img in val_images:
        copyfile(img, os.path.join(val_output_dir, os.path.basename(img)))

# Split and copy for colon cancer
split_and_copy(colonAcaDir, os.path.join(trainDir, 'colon_aca'), os.path.join(valDir, 'colon_aca'))
split_and_copy(colonBntDir, os.path.join(trainDir, 'colon_bnt'), os.path.join(valDir, 'colon_bnt'))

# Split and copy for lung cancer
split_and_copy(lungAcaDir, os.path.join(trainDir, 'lung_aca'), os.path.join(valDir, 'lung_aca'))
split_and_copy(lungBntDir, os.path.join(trainDir, 'lung_bnt'), os.path.join(valDir, 'lung_bnt'))
split_and_copy(lungSccDir, os.path.join(trainDir, 'lung_scc'), os.path.join(valDir, 'lung_scc'))

## Data Augmentation

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    '/kaggle/working/training_data',
    target_size=(224,224),
    class_mode='binary',
    batch_size=32
)

val_generator=val_datagen.flow_from_directory(
    '/kaggle/working/validation_data',
    target_size=(224,224),
    class_mode='binary',
    batch_size=32
)

Found 20000 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.


## Training the Data

In [7]:
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Input,InputLayer,GlobalAveragePooling1D,BatchNormalization

In [8]:
numClasses=len(train_generator.class_indices)

In [9]:
imageShape=(224,224,3)

In [53]:
from tensorflow.keras.regularizers import l2,l1
import tensorflow as tf

model = Sequential([
    Input(shape=imageShape),
    Conv2D(64,(3,3),activation='relu',kernel_regularizer=l2(0.1)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(16,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.1),
    Flatten(),
    Dense(128,activation='relu'),
    Dense(numClasses,activation='sigmoid')
])

In [54]:
from keras.optimizers import Adam,SGD

In [55]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [56]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 222, 222, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 111, 111, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 109, 109, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 52, 52, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 26, 26, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 26, 26, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 10816)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │     1,384,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,410,101 (5.38 MB)

 Trainable params: 1,410,101 (5.38 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
from keras.callbacks import EarlyStopping
earlystop= EarlyStopping(monitor='val_loss', patience=4,mode="max")

In [60]:
batch_size=32
steps_per_epoch = len(train_generator) // batch_size  
validation_steps = len(val_generator) // batch_size  
history = model.fit(train_generator,validation_data=val_generator,epochs=100,steps_per_epoch=steps_per_epoch,validation_steps=validation_steps,callbacks=[earlystop])

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.6440 - loss: 1.0255 - val_accuracy: 0.7188 - val_loss: 0.8669
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.7394 - loss: 0.8752 - val_accuracy: 0.7266 - val_loss: 0.7938
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.7615 - loss: 0.8182 - val_accuracy: 0.8047 - val_loss: 0.7778
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.7306 - loss: 0.7931 - val_accuracy: 0.7812 - val_loss: 0.7025
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.8089 - loss: 0.7240 - val_accuracy: 0.7969 - val_loss: 0.7677


In [61]:
# Evaluate on the training generator
train_results = model.evaluate(train_generator,batch_size=32)
print("Train loss, Train accuracy:", train_results)

# Evaluate on the validation generator
val_results = model.evaluate(val_generator)
print("Validation loss, Validation accuracy:", val_results)

625/625 ━━━━━━━━━━━━━━━━━━━━ 535s 855ms/step - accuracy: 0.7848 - loss: 0.8174
Train loss, Train accuracy: [0.8123947381973267, 0.7879999876022339]
157/157 ━━━━━━━━━━━━━━━━━━━━ 94s 596ms/step - accuracy: 0.7868 - loss: 0.7940
Validation loss, Validation accuracy: [0.8070391416549683, 0.7865999937057495]


In [62]:
def savingModel(model, model_name="model"): 
    model_filename = f"{model_name}.h5"
    model.save(model_filename)
    print(f"Model saved as {model_filename}")

In [63]:
savingModel(model,model_name="LungCancer")

Model saved as LungCancer.h5


In [64]:
from keras.models import load_model

In [65]:
def loadingModel(model_name="model"): 
    try:
        model = load_model(f"{model_name}.h5")
        model.summary()  # Print model summary if needed
        return model  # Return the loaded model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

In [66]:
loadedModel=loadingModel("LungCancer")

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 222, 222, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 111, 111, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 109, 109, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 52, 52, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 26, 26, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 26, 26, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 10816)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │     1,384,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,410,103 (5.38 MB)

 Trainable params: 1,410,101 (5.38 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [67]:
def loadImg(imgPath): 
    img = tf.keras.preprocessing.image.load_img(imgPath, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array=img_array/255.0
    return img_array

In [68]:
import tensorflow as tf

In [69]:
def predictImagesInFolder(testFolder, class_name, model_name): 
    savedModel = loadingModel(model_name)
    print(savedModel)

    if savedModel is None:
        print("Model loading failed.")
        return

    for subdir, dirs, files in os.walk(testFolder): 
        if subdir == testFolder: 
            continue
        trueLabel = os.path.basename(subdir)
        for file in files: 
            img_path = os.path.join(subdir, file)
            img = loadImg(img_path)
            
            # Debugging prediction probabilities
            predictions = savedModel.predict(img)
#             print(f"Prediction Probabilities for {file}: {predictions}")

            predicted_class_index = np.argmax(predictions, axis=1)
            predicted_class = class_name[predicted_class_index[0]]
            print(f"Predicted class: {predicted_class} File: {file}")

In [70]:
def testFolder(folder): 
    testFolder=f"{folder}"
    return testFolder

In [71]:
LungCancerTestFolder=testFolder("/kaggle/working/validation_data")

In [72]:
train_generator.class_indices

{'colon_aca': 0, 'colon_bnt': 1, 'lung_aca': 2, 'lung_bnt': 3, 'lung_scc': 4}

In [73]:
LungCancerClassname={0:'colon_aca', 1:'colon_bnt', 2:'lung_aca', 3:'lung_bnt',4:'lung_scc'}


In [ ]:
predictImagesInFolder(LungCancerTestFolder,LungCancerClassname,model_name="LungCancer")

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 222, 222, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 111, 111, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 109, 109, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 52, 52, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 26, 26, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 26, 26, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 10816)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │     1,384,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,410,103 (5.38 MB)

 Trainable params: 1,410,101 (5.38 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

<Sequential name=sequential_5, built=True>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
Predicted class: colon_aca File: colon_aca_1139.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted class: colon_bnt File: colon_aca_4765.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted class: colon_aca File: colon_aca_2332.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted class: colon_aca File: colon_aca_2720.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted class: colon_bnt File: colon_aca_3724.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted class: colon_bnt File: colon_aca_3764.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted class: colon_bnt File: colon_aca_3028.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted class: colon_bnt File: colon_aca_4542.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted class: colon_aca File: colon_aca_4580.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted class: colon_bnt File: colon_aca_0414.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted class: c